---
<center>

  # **Tarea 10: El átomo de Helio**

**Realizado Por:**

   Samuel Huertas Rojas

---
</center>

# 1.  
Use el método variacional con la base de estados 
$$
\Psi^{+} = \frac{1}{\sqrt{2}}\left( \lvert n_1\,00\,n_1' \,00 \rangle + \lvert n_1'\,00\,n_1\,00 \rangle \right)
$$
para obtener las correcciones radiales a la energía del estado fundamental. Hágalo para un máximo de 
$ n_1 $  y $ n_1' = 10 $.
(Resultado $ E_0 = -2.8752 $ u.a.).



Definición de la integrales directas y de intercambio: 

* Integral directa:

$$
\mathcal{I}_1 = \int_0^\infty dr_1 \, r_1^2 \, R_{n_1 0}(r_1) \, R_{n_2 0}(r_1) \int_0^\infty dr_2 \, r_2^2 \, R_{n_1' 0}(r_2) \, R_{n_2' 0}(r_2) \, \frac{1}{r_>}
$$

* Integral de intercambio:

$$
\mathcal{I}_2 = \int_0^\infty dr_1 \, r_1^2 \, R_{n_1 0}(r_1) \, R_{n_2' 0}(r_1) \int_0^\infty dr_2 \, r_2^2 \, R_{n_1' 0}(r_2) \, R_{n_2 0}(r_2) \, \frac{1}{r_>}
$$

In [1]:
import numpy as np
from scipy.integrate import dblquad
import sympy as sp
from functools import lru_cache
from sympy.physics.hydrogen import R_nl, E_nl

# ============================================================================
# CONFIGURACIÓN Y PARÁMETROS
# ============================================================================
ALPHA = 1 / 137.036
C = 1 / ALPHA

MU, E, HBAR = 1, 1, 1
Z = 2

FACTOR_CONVERSION = {"joul": 4.3598e-18, "cm": 219474.6313, "eV": 27.2114}

N1_MAX = 10
N2_MAX = 10

# Variables simbólicas
r1, r2 = sp.symbols("r1 r2", real=True, positive=True)


# ============================================================================
# FUNCIONES AUXILIARES
# ============================================================================
def factor_normalizacion(n1, n2, l1, l2, m1, m2):
    """
    Calcula el factor de normalización.

    Args:

    Returns:
        float: Factor de normalización
    """
    if n1 == n2 and l1 == l2 and m1 == m2:
        norma = 1 / 2
    else:
        norma = 1 / sp.sqrt(2)
    return norma


@lru_cache(maxsize=1000)
def calcular_integral_directa(n1, nl1, n2, nl2):
    """
    Calcula la integral directa I1 con caché para evitar recálculos.

    Args:
        n1, nl1, n2, nl2: Números cuánticos

    Returns:
        float: Valor de la integral
    """
    argumento = (
        (
            r1**2
            * r2**2
            * R_nl(nl2, 0, r2, Z)
            * R_nl(n2, 0, r1, Z)
            * R_nl(nl1, 0, r2, Z)
            * R_nl(n1, 0, r1, Z)
            / sp.Max(r1, r2)
        )
        * 2
        * factor_normalizacion(n1, nl1, 0, 0, 0, 0)
        * factor_normalizacion(n2, nl2, 0, 0, 0, 0)
    )

    fun = sp.lambdify((r1, r2), argumento)
    resultado, error = dblquad(fun, 0, 100, lambda x: 0, lambda x: 100)

    return resultado


@lru_cache(maxsize=1000)
def calcular_integral_intercambio(n1, nl1, n2, nl2):
    """
    Calcula la integral de intercambio I2 con caché.

    Args:
        n1, nl1, n2, nl2: Números cuánticos

    Returns:
        float: Valor de la integral
    """
    argumento = (
        (
            r1**2
            * r2**2
            * R_nl(nl2, 0, r1, Z)
            * R_nl(n1, 0, r1, Z)
            * R_nl(nl1, 0, r2, Z)
            * R_nl(n2, 0, r2, Z)
            / sp.Max(r1, r2)
        )
        * 2
        * factor_normalizacion(n1, nl1, 0, 0, 0, 0)
        * factor_normalizacion(n2, nl2, 0, 0, 0, 0)
    )

    fun = sp.lambdify((r1, r2), argumento)
    resultado, error = dblquad(fun, 0, 100, lambda x: 0, lambda x: 100)

    return resultado


# ============================================================================
# CONSTRUCCIÓN DE LA BASE DE ESTADOS
# ============================================================================
def construir_base_estados(n1_max, n2_max):
    """
    Construye la lista de estados base.

    Args:
        n1_max, n2_max: Valores máximos de n1 y n2

    Returns:
        tuple: (lista_estados, num_estados)
    """
    lista_estados = []
    contador = 0

    lista_n1 = []
    lista_n2 = []

    for n1 in range(1, n1_max + 1):
        for n2 in range(n1, n2_max + 1):
            lista_n1.append(n1)
            lista_n2.append(n2)
            lista_estados.append((n1, n2))
            contador += 1

    num_estados = len(lista_estados)
    # print(lista_estados)
    print(f"Número de estados para realizar el cálculo: {num_estados}")

    return lista_estados, lista_n1, lista_n2, num_estados


# ============================================================================
# CONSTRUCCIÓN DEL HAMILTONIANO
# ============================================================================
def construir_hamiltoniano_orden_cero(lista_estados, num_estados):
    """
    Construye la matriz del Hamiltoniano de orden cero (parte diagonal).

    Args:
        estados: Lista de tuplas (n1, n2)
        num_estados: Número total de estados

    Returns:
        np.ndarray: Matriz del Hamiltoniano H0
    """
    # Crear la matriz del hamiltoniano
    H0 = np.zeros((num_estados, num_estados))

    # Construccion de la matriz del hamiltoniano de orden cero
    for i in range(0, num_estados):
        n1, n2 = lista_estados[i]
        H0[i][i] = -0.5 * Z**2 * (1 / n1**2 + 1 / n2**2)

    autovalores_H0, autovectores_H0 = np.linalg.eig(H0)

    return H0

def agregar_perturbacion(H, estados):

    """
    Agrega las integrales directas y de intercambio al Hamiltoniano.

    Args:
        H: Matriz del Hamiltoniano (se modifica in-place)
        estados: Lista de estados
    """
    num_estados = len(estados)

    print("\nCalculando elementos de matriz...")
    total_elementos = num_estados * (num_estados + 1) // 2
    contador_elementos = 0

    for i in range(num_estados):
        n1_i, n2_i = estados[i]

        for j in range(i, num_estados):
            n1_j, n2_j = estados[j]

            # Calcular integrales
            I1 = calcular_integral_directa(n1_i, n2_i, n1_j, n2_j)
            I2 = calcular_integral_intercambio(n1_i, n2_i, n1_j, n2_j)

            # Actualizar matriz (aprovechando simetría, ya que es hermitica)
            H[i, j] = H[i, j] + I1 + I2
            H[j, i] = H[i, j]

            # Progreso
            contador_elementos += 1
            if contador_elementos % 10 == 0:
                porcentaje = 100 * contador_elementos / total_elementos
                print(
                    f"Progreso: {porcentaje:.1f}% ({contador_elementos}/{total_elementos})"
                )

            # print(f"<{n1_i},{n2_i}|{n1_j},{n2_j}> = {H[i, j]:.6f}")


# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================
def main():
    """
    Función principal que ejecuta el cálculo completo.
    """
    # Construir base de estados
    estados, lista_n1, lista_n2, num_estados = construir_base_estados(N1_MAX, N2_MAX)

    # Construir Hamiltoniano de orden cero
    H = construir_hamiltoniano_orden_cero(estados, num_estados)

    # Diagonalizar H0
    print("\nDiagonalizando Hamiltoniano de orden cero...")
    autovalores_H0, autovectores_H0 = np.linalg.eigh(H)
    print(f"\nAutovalores H0 (primeros 5):\n{autovalores_H0[:5]}")

    # Agregar perturbación
    agregar_perturbacion(H, estados)

    # Diagonalizar Hamiltoniano completo
    print("\nDiagonalizando Hamiltoniano completo...")
    autovalores, autovectores = np.linalg.eigh(H)

    print(f"\nAutovalores finales (primeros 5):\n{autovalores[:5]}")
    print(f"\nEnergía del estado fundamental: {autovalores[0]:.6f}")


In [6]:
main()

Número de estados para realizar el cálculo: 55

Diagonalizando Hamiltoniano de orden cero...

Autovalores H0 (primeros 5):
[-4.         -2.5        -2.22222222 -2.125      -2.08      ]

Calculando elementos de matriz...
Progreso: 0.6% (10/1540)
Progreso: 1.3% (20/1540)
Progreso: 1.9% (30/1540)
Progreso: 2.6% (40/1540)
Progreso: 3.2% (50/1540)
Progreso: 3.9% (60/1540)


c:\Users\samhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_quadpack_py.py:1260: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


Progreso: 4.5% (70/1540)
Progreso: 5.2% (80/1540)
Progreso: 5.8% (90/1540)
Progreso: 6.5% (100/1540)
Progreso: 7.1% (110/1540)
Progreso: 7.8% (120/1540)
Progreso: 8.4% (130/1540)
Progreso: 9.1% (140/1540)
Progreso: 9.7% (150/1540)
Progreso: 10.4% (160/1540)
Progreso: 11.0% (170/1540)
Progreso: 11.7% (180/1540)
Progreso: 12.3% (190/1540)
Progreso: 13.0% (200/1540)
Progreso: 13.6% (210/1540)
Progreso: 14.3% (220/1540)
Progreso: 14.9% (230/1540)
Progreso: 15.6% (240/1540)
Progreso: 16.2% (250/1540)
Progreso: 16.9% (260/1540)
Progreso: 17.5% (270/1540)
Progreso: 18.2% (280/1540)
Progreso: 18.8% (290/1540)
Progreso: 19.5% (300/1540)
Progreso: 20.1% (310/1540)
Progreso: 20.8% (320/1540)
Progreso: 21.4% (330/1540)
Progreso: 22.1% (340/1540)
Progreso: 22.7% (350/1540)
Progreso: 23.4% (360/1540)
Progreso: 24.0% (370/1540)
Progreso: 24.7% (380/1540)
Progreso: 25.3% (390/1540)
Progreso: 26.0% (400/1540)
Progreso: 26.6% (410/1540)
Progreso: 27.3% (420/1540)
Progreso: 27.9% (430/1540)
Progreso: 28.

c:\Users\samhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_quadpack_py.py:1260: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


Progreso: 70.8% (1090/1540)
Progreso: 71.4% (1100/1540)
Progreso: 72.1% (1110/1540)
Progreso: 72.7% (1120/1540)
Progreso: 73.4% (1130/1540)
Progreso: 74.0% (1140/1540)


c:\Users\samhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_quadpack_py.py:1260: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


Progreso: 74.7% (1150/1540)
Progreso: 75.3% (1160/1540)
Progreso: 76.0% (1170/1540)
Progreso: 76.6% (1180/1540)
Progreso: 77.3% (1190/1540)
Progreso: 77.9% (1200/1540)
Progreso: 78.6% (1210/1540)
Progreso: 79.2% (1220/1540)
Progreso: 79.9% (1230/1540)
Progreso: 80.5% (1240/1540)
Progreso: 81.2% (1250/1540)
Progreso: 81.8% (1260/1540)
Progreso: 82.5% (1270/1540)
Progreso: 83.1% (1280/1540)
Progreso: 83.8% (1290/1540)
Progreso: 84.4% (1300/1540)
Progreso: 85.1% (1310/1540)
Progreso: 85.7% (1320/1540)
Progreso: 86.4% (1330/1540)
Progreso: 87.0% (1340/1540)
Progreso: 87.7% (1350/1540)
Progreso: 88.3% (1360/1540)
Progreso: 89.0% (1370/1540)
Progreso: 89.6% (1380/1540)
Progreso: 90.3% (1390/1540)
Progreso: 90.9% (1400/1540)
Progreso: 91.6% (1410/1540)
Progreso: 92.2% (1420/1540)
Progreso: 92.9% (1430/1540)
Progreso: 93.5% (1440/1540)
Progreso: 94.2% (1450/1540)
Progreso: 94.8% (1460/1540)
Progreso: 95.5% (1470/1540)
Progreso: 96.1% (1480/1540)
Progreso: 96.8% (1490/1540)
Progreso: 97.4% (150

# 2.
Use ahora la base de estados singlete excitados discretos, es decir,
$
        \Psi^{+} = \frac{1}{\sqrt{2}}
        \left(
            \lvert 100\,n l m \rangle + \lvert n l m\,100 \rangle 
        \right)
$
hasta $n = 3 $ para obtener la energía del estado fundamental del Helio.
    (Resultado $ E_0 = 2.83865 $ u.a.). A partir de la matriz construida, muestre cuál es la energía perturbada a primer orden de los estados excitados.


Definición de la integrales directas y de intercambio: 

* Integral directa:

$$
\mathcal{I}_1 = \int_0^\infty dr_1 \, r_1^2 \, R_{1 0}(r_1) \, R_{1 0}(r_1) \int_0^\infty dr_2 \, r_2^2 \, R_{n' l'}(r_2) \, R_{n l}(r_2) \, \frac{1}{r_>} \delta_{l' l} \delta_{m' m}
$$

* Integral de intercambio:

$$
\mathcal{I}_2 = \dfrac{1}{2l +1} \int_0^\infty dr_1 \, r_1^2 \, R_{1 0}(r_1) \, R_{n l}(r_1) \int_0^\infty dr_2 \, r_2^2 \, R_{n' l'}(r_2) \, R_{1 0}(r_2) \, \frac{r_<^l}{r_>^{l+1}} \delta_{l' l} \delta_{m' m}
$$

In [58]:
@lru_cache(maxsize=1000)
def calcular_integral_directa2(n, l, m, np, lp, mp):  # noqa: E741
    """
    Calcula la integral directa I1 con caché para evitar recálculos.

    Args:
        n1, nl1, n2, nl2: Números cuánticos

    Returns:
        float: Valor de la integral
    """
    argumento = (
        (
            r1**2
            * r2**2
            * R_nl(1, 0, r1, Z)
            * R_nl(1, 0, r1, Z)
            * R_nl(np, lp, r2, Z)
            * R_nl(n, l, r2, Z)
            / sp.Max(r1, r2)
        )
        * sp.KroneckerDelta(lp, l)
        * sp.KroneckerDelta(mp, m)
        * factor_normalizacion(n, 1, l, 0, m, 0)
        * factor_normalizacion(np, 1, lp, 0, mp, 0)
    )

    fun = sp.lambdify((r1, r2), argumento)
    resultado, error = dblquad(fun, 0, 100, lambda x: 0, lambda x: 100)

    return resultado


@lru_cache(maxsize=1000)
def calcular_integral_intercambio2(n, l, m, np, lp, mp):  # noqa: E741
    """
    Calcula la integral de intercambio I2 con caché.

    Args:
        n1, nl1, n2, nl2: Números cuánticos

    Returns:
        float: Valor de la integral
    """
    argumento = (
        (
            r1**2
            * r2**2
            * R_nl(1, 0, r1, Z)
            * R_nl(n, l, r1, Z)
            * R_nl(np, lp, r2, Z)
            * R_nl(1, 0, r2, Z)
            * sp.Min(r1, r2) ** l
            / (sp.Max(r1, r2) ** (l + 1))
        )
        * sp.KroneckerDelta(lp, l)
        * sp.KroneckerDelta(mp, m)
        * factor_normalizacion(n, 1, l, 0, m, 0)
        * factor_normalizacion(np, 1, lp, 0, mp, 0)
    )

    fun = sp.lambdify((r1, r2), argumento)
    resultado, error = dblquad(fun, 0, 100, lambda x: 0, lambda x: 100)

    return resultado


def construir_hamiltoniano_orden_cero2(lista_estados, num_estados):
    """
    Construye la matriz del Hamiltoniano de orden cero (parte diagonal).

    Args:
        estados: Lista de tuplas (n1, n2)
        num_estados: Número total de estados

    Returns:
        np.ndarray: Matriz del Hamiltoniano H0
    """
    # Crear la matriz del hamiltoniano
    H0 = np.zeros((num_estados, num_estados))

    # Construccion de la matriz del hamiltoniano de orden cero
    for i in range(0, num_estados):
        n, l, m = lista_estados[i]  # noqa: E741
        H0[i][i] = E_nl(1, Z) + E_nl(n, Z)

    return H0


# ============================================================================
# CONSTRUCCIÓN DE LA BASE DE ESTADOS
# ============================================================================
def construir_base_estados2(n_max):
    """
    Construye la lista de estados base.

    Args:
        n_max: Valores máximos de n1 y n2

    Returns:
        tuple: (lista_estados, num_estados)
    """
    lista_estados = []

    for n in range(1, n_max + 1):
        for l in range(n):  # noqa: E741
            for m in range(-l, l + 1):
                lista_estados.append((n, l, m))

    num_estados = len(lista_estados)
    # print(lista_estados)
    print(f"Número de estados para realizar el cálculo: {num_estados}")

    return lista_estados, num_estados


def agregar_perturbacion_singleton_exitados(H, estados):
    """
    Agrega las integrales directas y de intercambio al Hamiltoniano.

    Args:
        H: Matriz del Hamiltoniano (se modifica in-place)
        estados: Lista de estados
    """
    num_estados = len(estados)

    print("\nCalculando elementos de matriz...")
    total_elementos = num_estados * (num_estados + 1) // 2
    contador_elementos = 0

    for i in range(num_estados):
        n, l, m = estados[i]  # noqa: E741

        for j in range(i, num_estados):
            np, lp, mp = estados[j]

            # Calcular integrales
            I1 = calcular_integral_directa2(n, l, m, np, lp, mp)
            I2 = calcular_integral_intercambio2(n, l, m, np, lp, mp)

            #print(f"I1 + I2={I1 + I2}")

            # Actualizar matriz (aprovechando simetría, ya que es hermitica)
            H[i, j] = H[i, j] +  2 * (I1 + I2) 
            H[j, i] = H[i, j]

            # Progreso
            contador_elementos += 1
            if contador_elementos % 10 == 0:
                porcentaje = 100 * contador_elementos / total_elementos
                print(
                    f"Progreso: {porcentaje:.1f}% ({contador_elementos}/{total_elementos})"
                )

            #print(f"<{n},{l},{m}|{np},{lp},{mp}> = {H[i, j]:.6f}")


# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================
def main2():
    """
    Función principal que ejecuta el cálculo completo.
    """
    # Construir base de estados
    estados, num_estados = construir_base_estados2(3)

    # Construir Hamiltoniano de orden cero
    H = construir_hamiltoniano_orden_cero2(estados, num_estados)

    # Diagonalizar H0
    print("\nDiagonalizando Hamiltoniano de orden cero...")
    autovalores_H0, autovectores_H0 = np.linalg.eigh(H)
    print(f"\nAutovalores H0 (primeros 5):\n{autovalores_H0[:5]}")

    # Agregar perturbación
    agregar_perturbacion_singleton_exitados(H, estados)

    # Diagonalizar Hamiltoniano completo
    print("\nDiagonalizando Hamiltoniano completo...")
    autovalores, autovectores = np.linalg.eigh(H)

    print(f"\nAutovalores finales (primeros 5):\n{autovalores[:5]}")
    print(f"\nEnergía del estado fundamental: {autovalores[0]:.5f}")


In [59]:
main2()

Número de estados para realizar el cálculo: 14

Diagonalizando Hamiltoniano de orden cero...

Autovalores H0 (primeros 5):
[-4.  -2.5 -2.5 -2.5 -2.5]

Calculando elementos de matriz...
Progreso: 9.5% (10/105)
Progreso: 19.0% (20/105)
Progreso: 28.6% (30/105)
Progreso: 38.1% (40/105)
Progreso: 47.6% (50/105)
Progreso: 57.1% (60/105)
Progreso: 66.7% (70/105)
Progreso: 76.2% (80/105)
Progreso: 85.7% (90/105)
Progreso: 95.2% (100/105)

Diagonalizando Hamiltoniano completo...

Autovalores finales (primeros 5):
[-2.83864846 -2.13619337 -2.11513518 -2.11513518 -2.11513518]

Energía del estado fundamental: -2.83865


# 3.
Haga lo mismo que en el punto anterior para obtener la energía perturbada a primer orden de los estados triplete.


In [92]:
def agregar_perturbacion_triplete_exitados(H, estados):
    """
    Agrega las integrales directas y de intercambio al Hamiltoniano.

    Args:
        H: Matriz del Hamiltoniano (se modifica in-place)
        estados: Lista de estados
    """
    num_estados = len(estados)

    print("\nCalculando elementos de matriz...")
    total_elementos = num_estados * (num_estados + 1) // 2
    contador_elementos = 0

    for i in range(num_estados):
        n, l, m = estados[i]  # noqa: E741
        for j in range(i, num_estados):
            np, lp, mp = estados[j]
            # Calcular integrales
            I1 = calcular_integral_directa2(n, l, m, np, lp, mp)
            I2 = calcular_integral_intercambio2(n, l, m, np, lp, mp)
            #print(f"I1 - I2={I1 - I2}")

            # Actualizar matriz (aprovechando simetría, ya que es hermitica)
            H[i, j] = H[i, j] + 2 *(I1 - I2)
            H[j, i] = H[i, j]

            # Progreso
            contador_elementos += 1
            if contador_elementos % 10 == 0:
                porcentaje = 100 * contador_elementos / total_elementos
                print(
                    f"Progreso: {porcentaje:.1f}% ({contador_elementos}/{total_elementos})"
                )

            print(f"<{n},{l},{m}|{np},{lp},{mp}> = {H[i, j]:.6f}")


# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================
def main3():
    """
    Función principal que ejecuta el cálculo completo.
    """
    # Construir base de estados
    estados, num_estados = construir_base_estados2(3)
    estados = estados[1:]
    num_estados = len(estados)

    # Construir Hamiltoniano de orden cero
    H1 = construir_hamiltoniano_orden_cero2(estados, num_estados)

    # Diagonalizar H0
    print("\nDiagonalizando Hamiltoniano de orden cero...")
    autovalores_H0, autovectores_H0 = np.linalg.eigh(H1)
    print(f"\nAutovalores H0 (primeros 5):\n{autovalores_H0[:5]}")

    # Agregar perturbación
    agregar_perturbacion_triplete_exitados(H1, estados)

    # Diagonalizar Hamiltoniano completo
    print("\nDiagonalizando Hamiltoniano completo...")
    autovalores, autovectores = np.linalg.eigh(H1)

    print(f"\nAutovalores finales (primeros 5):\n{autovalores[:5]}")
    print(f"\nEnergía del estado fundamental: {autovalores[0]:.6f}")

In [93]:
main3()

Número de estados para realizar el cálculo: 14

Diagonalizando Hamiltoniano de orden cero...

Autovalores H0 (primeros 5):
[-2.5        -2.5        -2.5        -2.5        -2.22222222]

Calculando elementos de matriz...
<2,0,0|2,0,0> = -2.124143
<2,0,0|2,1,-1> = 0.000000
<2,0,0|2,1,0> = 0.000000
<2,0,0|2,1,1> = 0.000000
<2,0,0|3,0,0> = 0.078607
<2,0,0|3,1,-1> = 0.000000
<2,0,0|3,1,0> = 0.000000
<2,0,0|3,1,1> = 0.000000
<2,0,0|3,2,-2> = 0.000000
Progreso: 11.0% (10/91)
<2,0,0|3,2,-1> = 0.000000
<2,0,0|3,2,0> = 0.000000
<2,0,0|3,2,1> = 0.000000
<2,0,0|3,2,2> = 0.000000
<2,1,-1|2,1,-1> = -2.116827
<2,1,-1|2,1,0> = 0.000000
<2,1,-1|2,1,1> = 0.000000
<2,1,-1|3,0,0> = 0.000000
<2,1,-1|3,1,-1> = 0.062275
<2,1,-1|3,1,0> = 0.000000
Progreso: 22.0% (20/91)
<2,1,-1|3,1,1> = 0.000000
<2,1,-1|3,2,-2> = 0.000000
<2,1,-1|3,2,-1> = 0.000000
<2,1,-1|3,2,0> = 0.000000
<2,1,-1|3,2,1> = 0.000000
<2,1,-1|3,2,2> = 0.000000
<2,1,0|2,1,0> = -2.116827
<2,1,0|2,1,1> = 0.000000
<2,1,0|3,0,0> = 0.000000
<2,1,0|3,

# 4.
Use ahora la base 
$
        \frac{1}{\sqrt{2}} \left(
            \lvert n\,00\,n' \,00 \rangle 
            + 
            \lvert n'\,00\,n\,00 \rangle
        \right)
$
y encuentre las energías de estados triplete.

In [6]:
def agregar_perturbacion_triplete(H, estados):

    """
    Agrega las integrales directas y de intercambio al Hamiltoniano.

    Args:
        H: Matriz del Hamiltoniano (se modifica in-place)
        estados: Lista de estados
    """
    num_estados = len(estados)

    print("\nCalculando elementos de matriz...")
    total_elementos = num_estados * (num_estados + 1) // 2
    contador_elementos = 0

    for i in range(num_estados):
        n1_i, n2_i = estados[i]

        for j in range(i, num_estados):
            n1_j, n2_j = estados[j]
            
            #if n1_i != n1_j and n2_i != n2_j:
            # Calcular integrales
            I1 = calcular_integral_directa(n1_i, n2_i, n1_j, n2_j)
            I2 = calcular_integral_intercambio(n1_i, n2_i, n1_j, n2_j)

            # Actualizar matriz (aprovechando simetría, ya que es hermitica)
            H[i, j] = H[i, j] + I1 - I2
            H[j, i] = H[i, j]

            # Progreso
            contador_elementos += 1
            if contador_elementos % 10 == 0:
                porcentaje = 100 * contador_elementos / total_elementos
                print(
                    f"Progreso: {porcentaje:.1f}% ({contador_elementos}/{total_elementos})"
                )

            print(f"<{n1_i},{n2_i}|{n1_j},{n2_j}> = {H[i, j]:.6f}")


# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================
def main4():
    """
    Función principal que ejecuta el cálculo completo.
    """
    # Construir base de estados
    estados, lista_n1, lista_n2, num_estados = construir_base_estados(4, 4)
    estados = estados[1:]
    num_estados = len(estados)

    # Construir Hamiltoniano de orden cero
    H = construir_hamiltoniano_orden_cero(estados, num_estados)

    # Diagonalizar H0
    print("\nDiagonalizando Hamiltoniano de orden cero...")
    autovalores_H0, autovectores_H0 = np.linalg.eigh(H)
    print(f"\nAutovalores H0 (primeros 5):\n{autovalores_H0[:5]}")

    # Agregar perturbación
    agregar_perturbacion_triplete(H, estados)

    # Diagonalizar Hamiltoniano completo
    print("\nDiagonalizando Hamiltoniano completo...")
    autovalores, autovectores = np.linalg.eigh(H)

    print(f"\nAutovalores finales (primeros 5):\n{autovalores[:5]}")
    print(f"\nEnergía del estado fundamental: {autovalores[0]:.6f}")


In [7]:
main4()

Número de estados para realizar el cálculo: 10

Diagonalizando Hamiltoniano de orden cero...

Autovalores H0 (primeros 5):
[-2.5        -2.22222222 -2.125      -1.         -0.72222222]

Calculando elementos de matriz...
<1,2|1,2> = -2.124143
<1,2|1,3> = 0.078607
<1,2|1,4> = 0.043163
<1,2|2,2> = 0.000000
<1,2|2,3> = 0.001214
<1,2|2,4> = 0.000982
<1,2|3,3> = 0.000000
<1,2|3,4> = 0.000115
<1,2|4,4> = 0.000000
Progreso: 22.2% (10/45)
<1,3|1,3> = -2.034783
<1,3|1,4> = 0.050332
<1,3|2,2> = 0.000000
<1,3|2,3> = 0.000640
<1,3|2,4> = 0.000524
<1,3|3,3> = 0.000000
<1,3|3,4> = 0.000061
<1,3|4,4> = 0.000000
<1,4|1,4> = -2.014413
<1,4|2,2> = 0.000000
Progreso: 44.4% (20/45)
<1,4|2,3> = 0.000409
<1,4|2,4> = 0.000336
<1,4|3,3> = 0.000000
<1,4|3,4> = 0.000039
<1,4|4,4> = 0.000000
<2,2|2,2> = -1.000000
<2,2|2,3> = 0.000000
<2,2|2,4> = 0.000000
<2,2|3,3> = 0.000000
<2,2|3,4> = 0.000000
Progreso: 66.7% (30/45)
<2,2|4,4> = 0.000000
<2,3|2,3> = -0.568946
<2,3|2,4> = 0.029755
<2,3|3,3> = 0.000000
<2,3|3,4> 